In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

tf.random.set_seed(777)  # for reproducibility

print(tf.__version__)


2.3.1


In [3]:
# ?? 뭐야 왜 없음?
# Data

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

# 전처리
def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

In [6]:
# W, b 선언, 0으로 초기화
W = tf.Variable(tf.zeros((2,1)), name = 'weight')
b = tf.Variable(tf.zeros((1, )), name = 'bias')
print('W = {}, b = {}'.format(W.numpy(), b.numpy()))

W = [[0.]
 [0.]], b = [0.]


In [7]:
# sigmoid 함수를 가설로 선언
def logistic_regression(features):
    hypothesis = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis

In [8]:
# 가설을 검증할 cost 함수 정의
def loss_fn(hypothesis, features, lables):
    cost = -tf.reduce_mean(labels * tf.math.log(logistic_regression(features)) + (1 - lables)
                           * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [9]:
# 0.5보다 크면 1, 작으면 0
def accuracy_fn(hypothesis, lables):
    predicted =tf.cast(hypothesis > 0.5, dtype = tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, lables), dtype=tf.float32))
    return accuracy

In [10]:
# GradientTape 통해 경사값 계산
def grad(hypothesis, features, lables):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features), features, lables)
    return tape.gradient(loss_value, [W, b])

In [12]:
# 학습하기
EPOCHS = 1001

for step in range(EPOCHS):
    for features, labels in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
        if step % 100 == 0:
            print('Iter: {}, Loss: {:.4f}'.format(step, loss_fn(logistic_regression(features),
                                                                features, labels)))
print('W = {}, B = {}'.format(W.numpy(), b.numpy()))
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(logistic_regression(x_data), y_data)
print('Testset Accuracy: {:.4f}'.format(test_acc))
            


Iter: 0, Loss: 0.6931
Iter: 100, Loss: 0.6931
Iter: 200, Loss: 0.6931
Iter: 300, Loss: 0.6931
Iter: 400, Loss: 0.6931
Iter: 500, Loss: 0.6931
Iter: 600, Loss: 0.6931
Iter: 700, Loss: 0.6931
Iter: 800, Loss: 0.6931
Iter: 900, Loss: 0.6931
Iter: 1000, Loss: 0.6931
W = [[0.]
 [0.]], B = [0.]
Testset Accuracy: 0.5000


이봐 이봐 결과 봐라 XOR을 logistic Regression 으로 풀어봄ㄴ 손실 값이줄어들지않고 계속 같아짐. W, b값은 처음에 우리가 설정해준 값이랑 똑같고,,정확도도 0.5밖에 안됨